# Hashtag stacked bar charts - Fake Covid-19 dataset

We've used the following packages:

In [1]:
import pandas as pd
import numpy as np
import json
import sys
import string
import re
import itertools  
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import emoji
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt 
import nltk
from PIL import Image
import altair as alt
import csv
import itertools

In order to do the classification of the tweets, we need to read the csv file and the json file:

In [2]:
csv_dataframe = pd.read_csv('dataset/FINAL_fakecovid_final_filtered_dataset_clean.csv',sep=";")
csv_dataframe['tweet_id'] = csv_dataframe['tweet_id'].astype(str)
csv_list = csv_dataframe.values.tolist()
lista_unica_csv=list(itertools.chain.from_iterable(csv_list))

data = []
with open('dataset/fakecovid_result_final_translated_full.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))


In this notebook we are going to visualize most frequent hashtags and we will classify them according to type of tweet that they're in.

In [3]:
index=0
cmt_list = []
#stopwords = set(STOPWORDS) 
for element in data:
    token_id = data[index]['id_str']
    indice_csv = lista_unica_csv.index(token_id)
    for entity in data[index]['entities']['hashtags']:
        entity['text'] = entity['text'].lower().replace(" ", "")
        token=entity['text']
        final_token = token + " " +lista_unica_csv[indice_csv+1].lower().replace(" ", "")
        cmt_list.append(final_token)
    index=index+1

We create the pandas DataFrame and then we work on it in order to create a better bar chart:

In [4]:
fdist = dict(nltk.FreqDist(cmt_list))


df = pd.DataFrame.from_dict(fdist, orient='index').reset_index()
df = df.rename(columns={'index':'hashtags', 0:'count'})
col_one_list = df['hashtags'].tolist()
col_two_list = df['count'].tolist()
typelist=[]
namelist=[]

index = 0

count_false = [0] * len(col_one_list)
count_part = [0] * len(col_one_list)

for el in col_one_list:
    tok = el.split()
    namelist.append(tok[0])
    if tok[0] in namelist:
        indx = namelist.index(tok[0])
        if tok[1] == "false":
            count_false[indx] = col_two_list[index]
        elif tok[1] == "partiallyfalse":
            count_part[indx] = col_two_list[index]
        else:
            print("errore count")
    index = index + 1

i=0
for el in col_two_list:
    col_two_list[i] = count_false[i] + count_part[i]
    i = i + 1

df['Hashtags']=namelist
df['False']=count_false
df['Partially False']=count_part
df['count']=col_two_list

df = df.sort_values(by=['count'],ascending=[False])
#print(df.head(20))

We visualize most frequent hashtags and we will classify them according to the type of tweet that they're in, we have filtered in the range of a minum of 10 tweets to a maxium to 212 tweets total (which is the maxium number of tweets created by a user in the dataset). Every bar is colored basing on the amount of tweet written per each category (False, Partually False).

In [5]:
range_ = ["#0C7BDC","#FFC20A"]


bars = alt.Chart(df).transform_fold(
    ['False', 'Partially False']
).mark_bar().encode(
    x=alt.X('key:N',title=None),
    y=alt.Y('value:Q',title="Tweet count"),
    color=alt.Color('key:N', scale=alt.Scale(range=range_),title="Category"),
    column="Hashtags:N"
).transform_filter(
    alt.FieldRangePredicate(field='count', range=[10, 212])
).properties(
    title="The most frequent hashtags classified by category (False, Partually False)", 
    width=75
).configure_title(
    fontSize=17,
    offset=25
).configure_axis(
    labelFontSize=13,
    titleFontSize=15,
    titlePadding=15
).configure_legend(
    titleFontSize=14,
    labelFontSize=12,
    titlePadding=10
).configure_header(
    titleFontSize=15,
    labelFontSize=12
)

bars

alt.Chart(...)

## Let's filter the Covid-related words

In [6]:
index_mentions=0
cmt_list_mentions = []
stopwords = ["covidstopswithme","coronavirusesp","coronavirusmadrid","beratulaacovid19","covid19sa","coronaupdatesindia","waspapovid19","estavirusloposome","covid19kenya","coronavirusreachesdelhi","cov19","coronaainazuilika","covid19espana","corona_snick","komeshacorona","nocoronabulos","covidreliefirs","covidsafe","coronavirusinindia","coronaupdatesinindia","covid19ethiopia","stopcovid","covid19outbreak","coronawarriors","covid19australia","kovid19","vaccine","coronacrisis","coronaviruskenya","covidiots","covid19uk","lockdownsa","covidiot","chinesevirus","pandemic","coronaviruslockdown","quarantinelife","sarscov2","coronalockdown","coronaupdate","covid19us","nomeat_nocoronavirus","covid19india","coronavirusupdate","quarantine","lockdown","chinavirus","coronaviruschina","coronavirusuk","wuhanvirus","coronavirusupdates","covid19pandemic","coronavirususa","covid19nigeria","coronaviruschina","coronarvirus","coronaoutbreak","ncov2019","2019ncov","chinacoronavirus","coronavirussa","wuhancoronavirus","coronarovirus","indiafightscorona","covid19", "coronavirus", "corona", "covid_19", "covid","coronavirusoutbreak","covid2019", "virus", "covid__19","covid19aus", "coronavirusindia","covidー19", "covid-19", "nocoronabulos.", "coronaviruspandemic"]
for element in data:
    token_id_mentions = data[index_mentions]['id_str']
    indice_csv_mentions = lista_unica_csv.index(token_id_mentions)
    for entity in data[index_mentions]['entities']['hashtags']:
        entity['text'] = entity['text'].lower().replace(" ", "")
        if entity['text'] not in stopwords:
            token_mentions=entity['text']
            final_token_mentions = token_mentions + " " +lista_unica_csv[indice_csv_mentions+1].lower().replace(" ", "")
            cmt_list_mentions.append(final_token_mentions)
    index_mentions=index_mentions+1


fdist_mentions = dict(nltk.FreqDist(cmt_list_mentions))
#print(fdist_mentions)
df_mentions = pd.DataFrame.from_dict(fdist_mentions, orient='index').reset_index()
df_mentions = df_mentions.rename(columns={'index':'hashtags', 0:'count'})
col_one_list_mentions = df_mentions['hashtags'].tolist()
col_two_list_mentions = df_mentions['count'].tolist()

#print(df_mentions)

typelist_mentions=[]
namelist_mentions=[]

indexm = 0

count_false_mentions = [0] * len(col_one_list_mentions)
count_part_mentions = [0] * len(col_one_list_mentions)

for el in col_one_list_mentions:
    tokm = el.split()
    namelist_mentions.append(tokm[0])
    #typelist.append(tok[1])
    if tokm[0] in namelist_mentions:
        indxm = namelist_mentions.index(tokm[0])
        if tokm[1] == "false":
            count_false_mentions[indxm] = col_two_list_mentions[indexm]
        elif tokm[1] == "partiallyfalse":
            count_part_mentions[indxm] = col_two_list_mentions[indexm]
        else:
            print("errore count")
   
    indexm = indexm + 1

im=0
for el in col_two_list_mentions:
    col_two_list_mentions[im] = count_false_mentions[im] + count_part_mentions[im]
    im = im + 1


df_mentions['Hashtags']=namelist_mentions
df_mentions['False']=count_false_mentions
df_mentions['Partially False']=count_part_mentions
df_mentions['count'] = col_two_list_mentions
#del df['count']

#df['type']=typelist

df_mentions = df_mentions.sort_values(by=['count'],ascending=[False])
#print(df_mentions.head(25))

Then we visualize the bar chart again:

In [7]:
range_ = ["#40B0A6","#E1BE6A"]

barsm = alt.Chart(df_mentions).transform_fold(
    ['False', 'Partially False'],
).mark_bar().encode(
    x=alt.X('key:N',title=None),
    y=alt.Y('value:Q',title="Tweet count"),
    color=alt.Color('key:N', scale=alt.Scale(range=range_),title="Category"),
    column="Hashtags:N"
).transform_filter(
    alt.FieldRangePredicate(field='count', range=[5, 27])
).properties(
    title="The most frequent hashtags classified by category (False, Partually False) WITHOUT COVID-RELATED WORDS",
    width=75
).configure_title(
    fontSize=17,
    offset=25
).configure_axis(
    labelFontSize=13,
    titleFontSize=15,
    titlePadding=15
).configure_legend(
    titleFontSize=14,
    labelFontSize=12,
    titlePadding=10
).configure_header(
    titleFontSize=15,
    labelFontSize=12
)

barsm

alt.Chart(...)